In [ ]:
from IPython.core.display import HTML
HTML("""
<style>

div.cell { /* Tunes the space between cells */
margin-top:1em;
margin-bottom:1em;
}

div.text_cell_render h1 { /* Main titles bigger, centered */
font-size: 2.2em;
line-height:1.4em;
text-align:center;
}

div.text_cell_render h2 { /*  Parts names nearer from text */
margin-bottom: -0.4em;
}


div.text_cell_render { /* Customize text cells */
font-family: 'Georgia';
font-size:1.2em;
line-height:1.4em;
padding-left:3em;
padding-right:3em;
}
</style>
""")


In [1]:
import pylab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# !pip install jenks

# Loading the data into a dataframe 

<b> Reading data from the required CSV files. </b>

In [2]:
flow = pd.read_csv("flow.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
occupancy = pd.read_csv("occupancy.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
speed = pd.read_csv("speed.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
timestamp = pd.read_csv("timestamp.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)

<b> Concating the values of the columns into a single dataframe </b>

In [3]:
df = pd.concat([flow,occupancy,speed,timestamp], axis=1)

In [4]:
df.columns = ['flow1','flow2', 'flow3','occupancy1','occupancy2', 'occupancy3','speed1','speed2', 'speed3','timestamp']
df.head()

,flow1,flow2,flow3,occupancy1,occupancy2,occupancy3,speed1,speed2,speed3,timestamp
0,9,4,255,4.0,2.0,4.0,67.0,60.0,0.0,2006-09-01T00:01:07
1,11,10,255,4.0,5.0,4.0,66.0,64.0,0.0,2006-09-01T00:02:07
2,-8,3,255,3.0,1.0,4.0,71.0,63.0,0.0,2006-09-01T00:03:07
3,5,9,255,2.0,3.0,4.0,65.0,70.0,0.0,2006-09-01T00:04:07
4,7,10,255,2.0,4.0,4.0,66.0,64.0,0.0,2006-09-01T00:05:08


<b> Implementing vector seperation. Dividing the single dataframe into seperate dataframes. </b>

In [5]:
df1 =  pd.DataFrame({"flow" : df.flow1, "occupancy" : df.occupancy1,"speed" : df.speed1,"timestamp" : df.timestamp,"detector" : "x"})
df2 = pd.DataFrame({"flow" : df.flow2, "occupancy" : df.occupancy2,"speed" : df.speed2,"timestamp" : df.timestamp,"detector" : "y"})
df3 = pd.DataFrame({"flow" : df.flow3, "occupancy" : df.occupancy3,"speed" : df.speed3,"timestamp" : df.timestamp,"detector" : "z"})

<b> Concatenating the dataframes </b>

In [6]:
frames = [df1, df2, df3]
result = pd.concat(frames)

<b> Sorting on timestamp </b>

In [7]:
result = result.sort_values("timestamp")

 # Cleaning the data

 <b>Finding outlier values and cleaning the occupancy column </b>
 
 Observations :-
        As we can see from the graph below, the occupancy values in the dataset increase consistently over a range
       of 0 to 100 but change drastically after that. This indicates possible outliers and wrong data.From the above
       observation, we conclude that even during a traffic jam the occupancy values should not be
       greater than a threshold of 120. So, we keep a variable jam_occupancy_threshold which holds a value of 120.
       This value helps us in identifying values which can be ignored while computing probablity distribution. We 
       can safely assume a probability density of 0 for rows carying such values. 
       
# Occupancy Outlier Graph
    

In [ ]:
val = np.sort(result["occupancy"].unique())
plt.plot(val)

<b>Finding outlier values and cleaning the Speed column </b>
 
Observations :- The graph below shows that the data is consistent till value 120 and gets skwed above that i.e speed > 120 mph. This makes perfect sense as speeds above are not practically feasible considering the speed limit laws.

# Speed Outlier Visualization

In [ ]:
plt.figure(figsize=(10,10))
plt.xlim(550, 650)
plt.plot(result["speed"].unique())

# Removing Wrong Data

<b> There are several rows which can be easily identified as wrong values. We will be identifying these values below and will work on removing this data </b>

<b> Observation :- </b> 

In the below cell we try to remove all rows where speed is less than 0 or the flow is less than zero. This is practically infeasible 

In [8]:
removeResult1 = result[(result['speed'] < 0) | (result['flow'] < 0)]

In the below cell we try to remove all rows where flow is equal to zero and speed is greater than zero. This is also practically not possible

In [9]:
removeResult2 = result[(result['flow'] == 0) & (result['speed'] > 0)]

In the below cell we work on defining the Jam occupancy threshold which we had earlier defined while checking occupancy outliers 

In [10]:
jam_occupancy_threshold = 120
removeResult3 = result[((result['speed'] == 0) & 
                            (result['flow'] == 0) & 
                            (result['occupancy'] != 0) & 
                            (result['occupancy'] < jam_occupancy_threshold))]

In the below cell we work on removing rows which have a speed of zero and an occupancy of zero but the flow is greater than zero. This is not possible practically

In [11]:
removeResult5 = result[(result['speed'] == 0) & (result['flow'] != 0) & (result["occupancy"] == 0)]

# Insignificant Data

In the below cell we work on defining the rows which have insignificant contribution to the data. This includes the condition where speed is zero, flow is zero and occupancy is also zero. This is a valid senario where there are no vehicles on the road. By running few observations we found more than 45,00,000 rows with such data in the first data set. Despite being correct, this data skews the probability distribution for all the other rows. Also, since we are interested in understanding the true outliers we can ignore these rows. By this step, we get a remarkable increase in the efficiency with which we can determine the outliers.

In [ ]:
removeResult4 = result[(result['speed'] == 0) & (result['flow'] == 0) & (result["occupancy"] == 0)]
removeResult6 = result[(pd.isnull(result['speed'])) | (pd.isnull(result['occupancy'])) | (pd.isnull(result['flow']))]

Finally removing all the wrong and insignificant data from the dataframe which would not be used for processing. 

In [12]:
result = pd.concat([result,removeResult1,removeResult1]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult2,removeResult2]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult3,removeResult3]).drop_duplicates(keep=False)
# result = pd.concat([result,removeResult4,removeResult4]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult5,removeResult5]).drop_duplicates(keep=False)
# result = pd.concat([result,removeResult6,removeResult6]).drop_duplicates(keep=False)

In [ ]:
result.head()

Below we are indexing the data based on the timestamp and detector column value. This helps us in understanding how the data can be processed further for probability density calculation

In [ ]:
res = result.set_index(["timestamp","detector"]).sort_index()
res.head()

# Probability Density Calculation # Approach 1

<h3> sampleData Function divides data into classes </h3>

In this function we initialize variable <i><b>"flowbins"</b></i> which would contain the bins obtained after dividing the dataset based on the specified <i><b>"threshold"</b></i>. We also use the <i><b>"columnName"</b></i> variable to specify the column for which we are putting the values into the bins. The data falling between the specified <i><b>threshold</b></i> would be put into the same bin.

<i> For e.g :- </i>

If we specify the <i><b>threshold</b></i> for <i><b>flow</b></i> column as <i><b>15</b></i>, then the flow column would be divided into <i><b>20</b></i> different bins in the manner 0-14, 14-29 ....
Similarly for all other columns we would have different values in <i><b>20</b></i> different bins based on <i><b>threshold</b></i>.

In [ ]:
from jenks import jenks
import numpy as np
def goodness_of_variance_fit(array, classes):
    # get the break points
    classes = jenks(array, classes)

    # do the actual classification
    classified = np.array([classify(i, classes) for i in array])

    # max value of zones
    maxz = max(classified)

    # nested list of zone indices
    zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

    # sum of squared deviations from array mean
    sdam = np.sum((array - array.mean()) ** 2)

    # sorted polygon stats
    array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]

    # sum of squared deviations of class means
    sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])

    # goodness of variance fit
    gvf = (sdam - sdcm) / sdam

    return gvf

def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

In [ ]:
result[result['speed'] == 0]

In [ ]:
result.head()

In [1]:
import sklearn
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from sklearn.utils import check_random_state

def sampleKMeans(data, columnName):
#     random_state = np.random.RandomState(0)
#     data = data.sort_values(columnName)
    column_names = ['flow', 'occupancy', 'speed']
    flowData = data[column_names]
    km = KMeans(n_clusters=8000, verbose=1).fit(flowData)
    return km

km = sampleKMeans(result, 'speed')

In [ ]:
from sklearn.neighbors.kde import KernelDensity

def sampleCluster(data, columnName):
    data = data.sort_values(columnName)
    flowData = data[columnName].reshape(-1,1)
    kde = KernelDensity(kernel='gaussian', bandwidth=3).fit(flowData)
    s = np.linspace(0,300)
    e = kde.score_samples(s.reshape(-1,1))
    plot(s, e)
    return s,e

s,e = sampleCluster(result, 'speed')

In [ ]:
# print result.loc((0))
test = result
test = test.reset_index(drop=True)

In [ ]:
# test.index.min()
# df1.index.max()

In [ ]:
print km.labels_
# {i: np.where(km.labels_ == i)[0] for i in range(km.n_clusters)}

# for i in range(km.n_clusters):
#     print np.where(km.labels_ == i)[0]

# for i in range(km.n_clusters):
#     print np.where(km.labels_ == i)[0].max()
for i in range(20):
    ids = np.where(km.labels_ == i)[0]
    df1 = pd.DataFrame(ids)
    df1.columns = ['id']
    df1.set_index('id')
    # df1.index
    joined = test.merge(df1, left_index=True, right_index=True, how = 'inner')
    print joined['speed'].mean()
# result.head()
# joined = df1.merge(result, how='inner')
# joined

In [ ]:
def sampleData(data, threshold, columnName):
    flowBins = np.zeros(shape=(20,), dtype = object)
    data = data.sort_values(columnName)
    difference = threshold
    high  = difference
    low = 0;
    for i in range(0, 20, 1):
        b = data[data[columnName] < high]
        b = b[b[columnName] >= low]
        flowBins[i] = b
        low = high
        high = high + difference
    return flowBins

Calling sampleData function for  <i><b>flow</b></i> column with a <i><b>threshold</b></i> value of <i><b>15</b></i>. This signifies that with a flow in range of <i><b>0 - 300</b></i> we would divide the flow column into <i><b>20 bins</b></i> containing datasets beween the specified range. 

In [ ]:
flowBins = np.zeros(shape=(20,), dtype = object)
flow_threshold = 15;
flowBins = sampleData(result, flow_threshold ,"flow")    

Calling sampleData function for  <i><b>occupancy</b></i> column with a <i><b>threshold</b></i> value of <i><b>6</b></i>. This signifies that with a subset of the orginal dataset, containing flow in range of <i><b>0 - 15</b></i> in first bin, we would further divide the first flow bin into <i><b>20</b></i> bins with a threshold of <i><b>6</b></i> for occupancy i.e  <i><b>flow_occupancy_bin[1]</b></i> will contain 20 bins with each bin conatining values between 0-6, 7-14 ... 

In [ ]:
flow_occupancy_bin = np.ndarray(shape=(20,20), dtype = object)
occupancy_threshold = 6;
for i in range(0,20,1):
    data = pd.DataFrame(flowBins[i])
    flow_occupancy_bin[i] = sampleData(data, 6 ,"occupancy") 

<h5> Finally .... </h5>
<br>
Calling sampleData function for  <i><b>speed</b></i> column with a <i><b>threshold</b></i> value of <i><b>13</b></i>. This signifies that with a subset of the orginal dataset, containing flow in range of <i><b>0 - 15</b></i> in first bin, and occupancy in the first bin in the range of <i><b>0 - 6</b></i>, we would further divide the first flow_occupancy_bin into <i><b>20</b></i> bins with a threshold of <i><b>6</b></i> for speed i.e  <i><b>flow_occupancy_speed_bin[1]</b></i> will contain 20 bins for speed with each bin conatining values between 0-13, 14-28 ... 

In [ ]:
import numpy as np
flow_occupancy_speed_bin = np.ndarray(shape=(20,20,20), dtype = object)
speed_threshold = 13;

for i in range(0,20,1):
    for j in range(0,20,1):
        data = flow_occupancy_bin[i][j];
        flow_occupancy_speed_bin[i][j] = sampleData(data, 13 ,"speed") 

# Iterating the flow_occupancy_speed_bin to compute probability density

<h4> Possibly the most important step.... </h4>

<br> In this step we find the probability density of the bins using the formula 

   <br><center><b><i> probability = points_inside_box / float(total_points * volume) </i></b></center>
   <br> We iterate over the total number of available bins i.e 20 bins for flow, 20 bins for occupancy and 20 bins for speed to calculate total points_inside_box. Further we have already calculated the volume of each box i.e <b><i> volume = flow_threshold </i>x <i>occupancy_threshold</i> x <i>speed_threshold </i></b>. The total rows in the dataset is being calculated using <b><i> total_points = len(result) </i></b> which we plug into the above formula. This calculates the probability of each bin.
   
   <br> NOW...WE FACED PERFORMANCE ISSUES.....  :( :( 
   <br> WHAT TO DO NOW....
   <br><br> The issue occoured while we were trying to merge back the probability column into a new dataframe to create our final result. It took almost half-hour to run through a single bin and merge it back to our existing dataframe. This way it would have taken days to complete this dataset. We searched and found this to be a drawback of dataframes.
   http://stackoverflow.com/questions/31860671/pandas-append-perfomance-concat-append-using-larger-dataframes
   
   <br> THEN CAME OUR SAVIOUR....TERMINAL :) :) 
   <br><br> We started saving all the bins to the file system i.e in CSV files. Using the capability of IPython notebook to run commands on command line directly we created a new directory and saved all our output to that. Next we merged all the files into one file <b><i> merged.csv using terminal cat command</i></b>. Then we delete the output folder containing all the intermediate files. 
   <br> <b> This way we completed our file operations in 2-3 mins. :) </b>
   
   

In [ ]:
!mkdir output

In [ ]:
final_result = pd.DataFrame(columns = ["detector","flow","occupancy","speed","timestamp", "probability"])
total_points = len(result)
volume = flow_threshold * occupancy_threshold * speed_threshold
for i in range(0,20,1):
    for j in range(0,20,1):
        for k in range(0,20,1):
            prob_data = flow_occupancy_speed_bin[i][j][k]
            points_inside_box = prob_data.shape[0]
            probability = points_inside_box/float(total_points * volume)
            prob_data["probability"] = probability
            prob_data.to_csv("output/out" + str(i) + '-' + str(j) + '-' + str(k) + ".csv",sep="\t")       

In [ ]:
! cat output/*.csv > merged.csv

In [ ]:
!rm -rf output/

In [ ]:
merged = pd.read_csv("merged.csv", na_values=['-'], delimiter="\t", error_bad_lines=False)

In [ ]:
removeHeader = merged[merged["probability"] == "probability"]

In [ ]:
merged = pd.concat([merged,removeHeader,removeHeader]).drop_duplicates(keep=False)

In [ ]:
merged.tail()

In [ ]:
merged["probability"] = merged["probability"].apply(pd.to_numeric)
merged = merged.sort_values(["probability"])

final_columns = ['flow', 'speed', 'occupancy', 'probability']
merged = merged[final_columns]
final_result = pd.DataFrame(columns = ["flow","speed","occupancy", "probability"])

for i in range(0,len(merged),100):
    final_result = final_result.append(merged.iloc[[i]])
final_result.to_csv("1160.txt",sep="\t")

In [ ]:
merged